# Initial Workflow

In [88]:
# Importing Libraries
import os
import re
from dotenv import load_dotenv

# LangChain Core
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LLM Models
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# RAGAS Evaluation Framework
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import (
    AspectCritic,
    RubricsScore,
    ContextPrecision,
    Faithfulness
)
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset


# Load environment variables from .env file
load_dotenv()


True

In [89]:
# Important: Please set your own password and database schema name from .env file
db_user = os.environ.get("DB_USERNAME") # It is usually 'root' for local MySQL installations
db_password = os.environ.get("DB_PASSWORD") # Set during MySQL installation


# Connect MySQL database
host = 'localhost'
port = '3306'
username = 'root'
password = db_password
database_schema = 'text_to_sql'
#mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
mysql_uri = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=2)

In [90]:
# Create the LLM Prompt Template

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query do not include anything else. Provide me sql query in a single line do not add line breaks.
Table Schema: {schema}
Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [91]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [92]:
# LLM Initialization
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = os.environ.get("GOOGLE_API_KEY"))

In [93]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema=lambda _:get_schema(db)) 
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)



In [94]:
# test the SQL query chain with a sample question

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)

```sql
SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = 'Geiss Company'
```


In [95]:
query = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE) # For removing ```sql ``` from the response

if query:
    query=query.group(1).strip()

query

"SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = 'Geiss Company'"

In [96]:
# Execute the generated SQL query on the database
print(db.run(query))

[(5516846.999999994,)]


The generated answer matches our result when the same SQL query is run on MySQL Workbench

# RAGAs(Retrieval-Augmented Generation Assessment) Implementation 

In [97]:
llm = ChatAnthropic(
    model="claude-3-opus-20240229",  # or "claude-3-sonnet-20240229", "claude-3-haiku-20240307"
    anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY")
)

In [98]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [99]:
evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings)

C:\temp\ipykernel_29964\108186089.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = LangchainLLMWrapper(llm)
C:\temp\ipykernel_29964\108186089.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings)


In [100]:
maliciousness_definition = (
    "Is the response intended to harm, deceive, or exploit users?"
)

aspect_critic = AspectCritic(
    name="maliciousness",
    definition=maliciousness_definition,
    llm=evaluator_llm,
)

# adapeted google's helpfulness_prompt_template
helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information, and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics, llm=evaluator_llm)

In [101]:
context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

In [102]:
user_inputs = [
    "What was the budget of Product 12",
    "What are the names of all products in the products table?",
    "List all customer names from the customers table.",
    "Find the name and state of all regions in the regions table.",
    "What is the name of the customer with Customer Index = 1"
]

responses = []
retrieved_contexts = []

for question in user_inputs:
    resp = sql_chain.invoke({"question": question})
    

    match = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)
    if match:
        query = match.group(1).strip()
        responses.append(query)
        retrieved_contexts.append(query)

In [103]:
responses

["SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'",
 'SELECT `Product Name` FROM products',
 'SELECT `Customer Names` FROM customers',
 'SELECT name, state FROM regions',
 'SELECT `Customer Names` FROM Customers WHERE `Customer Index` = 1']

In [104]:
references=[
"SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12';",
"SELECT `Product Name`ROM products;",
"SELECT `Customer Names`FROM customers;",
"SELECT name, state FROM regions;",
"SELECT `Customer Names` FROM customers WHERE `Customer Index` = 1;"
]

In [ ]:
samples = []
for i in range(len(user_inputs)):

    sample = SingleTurnSample(
        user_input=user_inputs[i],
        retrieved_contexts=list(retrieved_contexts),
        response=responses[i],
        reference=references[i],
    )
    samples.append(sample)

In [107]:
samples

[SingleTurnSample(user_input='What was the budget of Product 12', retrieved_contexts=["SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'", 'SELECT `Product Name` FROM products', 'SELECT `Customer Names` FROM customers', 'SELECT name, state FROM regions', 'SELECT `Customer Names` FROM Customers WHERE `Customer Index` = 1'], reference_contexts=None, retrieved_context_ids=None, reference_context_ids=None, response="SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'", multi_responses=None, reference="SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12';", rubrics=None),
 SingleTurnSample(user_input='What are the names of all products in the products table?', retrieved_contexts=["SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'", 'SELECT `Product Name` FROM products', 'SELECT `Customer Names` FROM customers', 'SELECT name, state FROM regions', 'SELECT `Customer Names` FROM Customers 

In [108]:
ragas_eval_dataset = EvaluationDataset(samples=samples)
ragas_eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What was the budget of Product 12,[SELECT `2017 Budgets` FROM `2017_budgets` WHE...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...
1,What are the names of all products in the prod...,[SELECT `2017 Budgets` FROM `2017_budgets` WHE...,SELECT `Product Name` FROM products,SELECT `Product Name`ROM products;
2,List all customer names from the customers table.,[SELECT `2017 Budgets` FROM `2017_budgets` WHE...,SELECT `Customer Names` FROM customers,SELECT `Customer Names`FROM customers;
3,Find the name and state of all regions in the ...,[SELECT `2017 Budgets` FROM `2017_budgets` WHE...,"SELECT name, state FROM regions","SELECT name, state FROM regions;"
4,What is the name of the customer with Customer...,[SELECT `2017 Budgets` FROM `2017_budgets` WHE...,SELECT `Customer Names` FROM Customers WHERE `...,SELECT `Customer Names` FROM customers WHERE `...


In [109]:
from ragas import evaluate

ragas_metrics = [ context_precision, rubrics_score]

result = evaluate(
    metrics=ragas_metrics,
    dataset=ragas_eval_dataset
)
result

Evaluating:  50%|█████     | 5/10 [02:42<04:12, 50.46s/it]Exception raised in Job[8]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Evaluating: 100%|██████████| 10/10 [03:00<00:00, 18.00s/it]


{'context_precision': 1.0000, 'helpfulness': 4.5000}